In [18]:
!pip install dotenv
!pip install transformers accelerate
!pip install streamlit
!npm install localtunnel

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 755ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

In [19]:
%%writefile CodeModel.py

import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from accelerate import init_empty_weights, infer_auto_device_map, load_checkpoint_in_model
import torch
import time
from huggingface_hub import login

# (replace with your hugging face token)
login("HF_Token")

def generate_code(prompt):
    model_id = "Salesforce/codegen-350M-multi"
    tokenizer = AutoTokenizer.from_pretrained(model_id)


    with init_empty_weights():
        model = AutoModelForCausalLM.from_pretrained(model_id)
    device_map = infer_auto_device_map(model, no_split_module_classes=["LlamaDecoderLayer"],
                                       max_memory={"cpu": "48GiB"})
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map=device_map,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map=device_map,
        torch_dtype=torch.float16
    )

    start = time.time()
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=128
    )
    result = ''.join([s['generated_text'] for s in sequences])
    print(result)
    end = time.time()
    return result, start, end


Overwriting CodeModel.py


In [20]:
from CodeModel import generate_code
print(generate_code('fibonacci sequence'), sep='\n')


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (

fibonacci sequence
    fib = fibonacci(n)
    for i in range(n):
        fib[i] = fib[i] + fib[i-1]
    return fib

def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    elif n == 2:
        return fibonacci(n-1)
    else:
        return fibonacci(n-1) + fibonacci(n-2)

def fibonacci_seq(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    elif n == 2:
        return fibonacci_seq(n-1)
    else:
        return fibonacci_seq(n-1) + fibonacci_seq(n-2)

def fibonacci_seq_seq(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    elif n == 2:
        return fibonacci_seq_seq(
('fibonacci sequence\n    fib = fibonacci(n)\n    for i in range(n):\n        fib[i] = fib[i] + fib[i-1]\n    return fib\n\ndef fibonacci(n):\n    if n == 0:\n        return 0\n    elif n == 1:\n        return 1\n    elif n == 2:\n        return fibonacci(n-1)\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n\ndef fibonacci_seq(n):\n 

In [21]:
%%writefile app.py

import streamlit as st
import time
from CodeModel import generate_code

st.header("CodeGenie: AI-powered code generator")

with st.form("my_form"):
    user_input = st.text_area("Enter your text prompt below and click the button to submit.")
    submit = st.form_submit_button(label="Submit text prompt")

if submit:
    with st.spinner(text="Generating code... It may take some time"):
        code, start, end = generate_code(prompt=user_input)
        hours, rem = divmod(end - start, 3600)
        minutes, seconds = divmod(rem, 60)

        st.success(
            "Processing time: {:0>2}:{:0>2}:{:05.2f}.".format(
                int(hours), int(minutes), seconds
            )
        )
        st.code(code, language='python')

    st.sidebar.markdown("## Guide")
    st.sidebar.info("This tool uses Salesforce CodeGen 350M model.")


Overwriting app.py


In [22]:
!wget -q -O - ipv4.icanhazip.com

34.127.53.224


In [23]:
!pip install streamlit -q
!wget -q -O - ipv4.icanhazip.com

!streamlit run app.py & npx localtunnel --port 8501


34.127.53.224


⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.53.224:8501

your url is: https://six-teams-begin.loca.lt
2025-06-19 16:03:39.493751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750349019.580904   12377 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750349019.602934   12377 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Some weights of the model checkpoint at Salesforce/codegen-350M-multi were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'tran